In [ ]:
from PIL import Image
import numpy as np
import sys
 
 
# load the image and convert into
# numpy array
img = Image.open('IMG_8201.jpeg')
 
# asarray() class is used to convert
# PIL images into NumPy arrays
numpydata = np.asarray(img)
 
#  shape

R = np.zeros((480, 640), dtype = np.uint8)
G = np.zeros((480, 640), dtype = np.uint8)
B = np.zeros((480, 640), dtype = np.uint8)
for i in range(480):
    for j in range(640):
        R[i][j] = numpydata[i][j][0]
        G[i][j] = numpydata[i][j][1]
        B[i][j] = numpydata[i][j][2]
# RGB2YCrCb
Y = np.zeros((480, 640), dtype = np.uint8)
for i in range(480):
    for j in range(640):
       Y[i][j] = np.round(0.299*R[i][j] + 0.587*G[i][j] + 0.145*B[i][j])

img = Image.fromarray(Y, "L")
print(R.shape)
img.save("test_Y.jpg")

In [213]:
def convolve2D(image, kernel, padding=0, strides=1):
    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return output

In [ ]:
# Sobel filter processing
kernel_X = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
kernel_Y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
# kernel_Z = np.array([[0.25, 0.25], [0.25, 0.25]])

output_X = convolve2D(Y, kernel_X, 1, 1)
output_Y = convolve2D(Y, kernel_Y, 1, 1)
output = np.abs(output_X)+np.abs(output_Y)
output = np.floor((output/np.max(output))*(2**8))
# output = convolve2D(output, kernel_Z, 1, 1)
# print(output)

output_matrix = np.zeros((480, 640), dtype = np.uint8)

th = 50
for i in range(480):
    for j in range(640):
        if(output[i][j] > th):
            output_matrix[i][j] = 255
        else:
            output_matrix[i][j] = 0

img = Image.fromarray(output_matrix, "L")
img.save("Sobel.jpg")
np.savetxt("bar.txt", output_matrix, fmt="%3d")


In [ ]:
#template matching

import matplotlib.pyplot as plt
#read in img

template_img = Image.open('Sobel_crop.jpg')
template_data = np.asarray(template_img)

# do convolution
conv_out = convolve2D(output_matrix, template_data)
print(conv_out.shape)


In [ ]:
# pooling and grouping via blob detection

def sum(x, y, step, output_matrix):
    accumu = 0
    for i in range(step):
        for j in range(step):
            accumu += output_matrix[x*step+i][y*step+j]
    return accumu

step = 8
out_matr = np.zeros((int(output_matrix.shape[0]/step), int(output_matrix.shape[1]/step)))
for i in range(int(output_matrix.shape[0]/step)):
    for j in range(int(output_matrix.shape[1]/step)):
        out_matr[i][j] = sum(i, j, step, output_matrix)/255

print(out_matr)
output_matrix_2 = np.zeros(out_matr.shape)
thr = 15
for i in range(out_matr.shape[0]):
    for j in range(out_matr.shape[1]):
        if out_matr[i][j]>thr:
            output_matrix_2[i][j] = 1
        else:
            output_matrix_2[i][j] = 0
            
np.savetxt("dump1.txt", output_matrix_2, fmt = "%d")
np.savetxt("dump2.txt", out_matr, fmt = "%10d")